In [11]:
using NonNegLeastSquares
using BenchmarkTools

In [58]:
include("nnls.jl")

NNLS

In [59]:
@code_warntype NNLS.construct_householder!(rand(5), 1.0)

Variables:
  #self#::NNLS.#construct_householder!
  u::Array{Float64,1}
  up::Float64
  m::Int64
  clinv::Float64
  sm::Float64
  cl::Float64
  result::Float64
  #temp#::Int64
  ui::Float64
  fx@_11::Float64
  fx@_12::Float64

Body:
  begin 
      NewvarNode(:(clinv::Float64))
      NewvarNode(:(sm::Float64))
      NewvarNode(:(cl::Float64))
      NewvarNode(:(result::Float64))
      m::Int64 = (Base.arraylen)(u::Array{Float64,1})::Int64 # line 5:
      unless (Base.sle_int)(m::Int64,1)::Bool goto 10 # line 6:
      return up::Float64
      10:  # line 9:
      cl::Float64 = $(Expr(:invoke, LambdaInfo for _mapreduce(::Base.#abs, ::Base.#scalarmax, ::Base.LinearFast, ::Array{Float64,1}), :(Base._mapreduce), :(NNLS.abs), :(Base.scalarmax), :($(Expr(:new, :(Base.LinearFast)))), :(u))) # line 10:
      # meta: location operators.jl > 64
      # meta: location float.jl < 323
      fx@_11::Float64 = (Base.box)(Float64,(Base.sitofp)(Float64,0))
      # meta: pop location
      # meta: pop loc

In [60]:
@code_warntype NNLS.apply_householder!(rand(5), 1.0, rand(5))

Variables:
  #self#::NNLS.#apply_householder!
  u::Array{Float64,1}
  up::Float64
  c::Array{Float64,1}
  m::Int64
  cl::Float64
  b::Float64
  sm::Float64
  #temp#@_9::Int64
  i@_10::Int64
  #temp#@_11::Int64
  i@_12::Int64
  fx@_13::Float64
  fx@_14::Float64
  fy::Float64

Body:
  begin 
      NewvarNode(:(cl::Float64))
      NewvarNode(:(b::Float64))
      NewvarNode(:(sm::Float64))
      m::Int64 = (Base.arraylen)(u::Array{Float64,1})::Int64 # line 28:
      unless (Base.sle_int)(m::Int64,1)::Bool goto 9 # line 29:
      return
      9:  # line 32:
      cl::Float64 = (Base.box)(Base.Float64,(Base.abs_float)((Base.arrayref)(u::Array{Float64,1},1)::Float64)) # line 33:
      # meta: location operators.jl > 64
      # meta: location float.jl < 323
      fx@_13::Float64 = (Base.box)(Float64,(Base.sitofp)(Float64,0))
      # meta: pop location
      # meta: pop location
      unless (Base.box)(Base.Bool,(Base.or_int)((Base.lt_float)(fx@_13::Float64,cl::Float64)::Bool,(Base.box)(Base.Bo

In [61]:
srand(1)
m = 100
n = 200
A = randn(m, n)
b = randn(m)
x1 = NonNegLeastSquares.nnls(A, b);
x2 = NNLS.nnls(A, b);
@assert x1 ≈ x2

In [62]:
work = NNLS.NNLSWorkspace(Float64, size(A, 1), size(A, 2))
@code_warntype NNLS.nnls!(work, copy(A), copy(b), 3*size(A, 2))

Variables:
  #self#::NNLS.#nnls!
  work::NNLS.NNLSWorkspace{Float64,Int64}
  A::Array{Float64,2}
  b::Array{Float64,1}
  itermax::Int64
  factor::Float64
  x::Array{Float64,1}
  w::Array{Float64,1}
  zz::Array{Float64,1}
  idx::Array{Int64,1}
  #temp#@_11::Int64
  m::Int64
  n::Int64
  iter::Int64
  izmax::Int64
  iz2::Int64
  iz1::Int64
  iz::Int64
  j::Int64
  nsetp::Int64
  npp1::Int64
  up::Float64
  terminated::Bool
  sm::Float64
  #temp#@_25::Int64
  #temp#@_26::Int64
  i@_27::Int64
  jj::Int64
  #temp#@_29::Int64
  ip@_30::Int64
  i@_31::Int64
  alpha::Float64
  i@_33::Int64
  allfeasible::Bool
  #temp#@_35::Int64
  #temp#@_36::Int64
  ii::Int64
  cc::Float64
  ss::Float64
  sig::Float64
  #temp#@_41::Int64
  temp::Float64
  #temp#@_43::Int64
  l@_44::Int64
  #temp#@_45::Int64
  ip@_46::Int64
  l@_47::Int64
  #temp#@_48::Int64
  t::Float64
  ip@_50::Int64
  l@_51::Int64
  #temp#@_52::Int64
  l@_53::Int64
  #temp#@_54::Int64
  jz::Int64
  wmax::Float64
  Asave::Float64
  unorm::F

In [ ]:
@benchmark NonNegLeastSquares.nnls($A, $b)

In [ ]:
@benchmark NNLS.nnls($A, $b)

In [ ]:
@benchmark NNLS.nnls!(w1, A1, b1) setup=(w1 = NNLS.NNLSWorkspace(Float64, m, n); A1 = copy(A); b1 = copy(b))

In [ ]:
@benchmark NNLS.nnls!(work, A, b)

In [14]:
3.681 / (49.1e-3)

74.969450101833

In [1]:
macro wrappedallocs(expr)
    argnames = [gensym() for a in expr.args]
    quote
        function g($(argnames...))
            @allocated $(Expr(expr.head, argnames...))
        end
        $(Expr(:call, :g, [esc(a) for a in expr.args]...))
    end
end

@wrappedallocs (macro with 1 method)

In [3]:
# SUBROUTINE H12 (MODE,LPIVOT,L1,M,U,IUE,UP,C,ICE,ICV,NCV) 

In [33]:
u = normalize([1, 1])
lpivot = 1
up = construct_householder!(lpivot, u)
@show u up
c = [1., 0]
apply_householder!(lpivot, u, up, c)
c

u = [-1.0,0.707107]
up = 1.7071067811865475


2-element Array{Float64,1}:
 -0.707107
 -0.707107

In [30]:
v = normalize([1., 1])

2-element Array{Float64,1}:
 0.707107
 0.707107

In [24]:
x = [1., 0]

2-element Array{Float64,1}:
 1.0
 0.0

In [25]:
x - 2 * v * v' * x

2-element Array{Float64,1}:
  2.22045e-16
 -1.0        

In [28]:
u

2-element Array{Float64,1}:
 -1.0     
  0.707107

In [27]:
c = [1., 0]
apply_householder!(lpivot, u, up, c)
c

2-element Array{Float64,1}:
 -0.707107
 -0.707107

In [19]:
function h12{T}(mode::Integer, lpivot::Integer, l1::Integer, m::Integer, u::Matrix{T}, up::T, C::Matrix{T})
    @assert size(u, 2) == m
    ncv = size(C, 2)
    icv = size(C, 1)
    ice = 1
    
    if 0 >= lpivot || lpivot >= l1 || l1 > m
        return
    end
    
    cl = abs(u[1, lpivot])
    
    if mode == 2
        if cl <= 0
            return
        end
    else
        # ****** CONSTRUCT THE TRANSFORMATION. ******
        for j in l1:m
            cl = max(abs(u[1, j]), cl)
        end
        
        if cl <= 0
            return
        end
        
        clinv = 1 / cl
        sm = (u[1, lpivot] * clinv)^2
        for j in l1:m
            sm += (u[1, j] * clinv)^2
        end
        cl *= √sm
        if u[1, lpivot] > 0
            cl = -cl
        end
        up = u[1, lpivot] - cl
        u[1, lpivot] = cl
    end
    
    # ****** APPLY THE TRANSFORMATION  I+U*(U**T)/B  TO C. ******
    if ncv <= 0
        return
    end
    
    b = up * u[1, lpivot]
    
    # B  MUST BE NONPOSITIVE HERE.  IF B = 0., RETURN.
    if b >= 0
        return
    end
    
    b = 1 / b
    ic2 = 1 - icv + ice * (lpivot - 1)
    incr = ice * (l1 - lpivot)
    for j in 1:ncv
        i2 += icv
        i3 = i2 + incr
        i4 = i3
        sm = c[i2] * up
        for i in l1:m
            sm += c[i3] * u[1, i]
            i3 += ice
        end
        if sm != 0
            sm *= b
            c[i2] = c[i2] + sm * up
            for i in l1:m
                c[i4] += sm * u[1, i]
                i4 += ice
            end
        end
    end                
end

h12 (generic function with 1 method)

In [17]:


function nnls!{T}(work::NNLSWorkspace{T}, A::Matrix{T}, b::Vector{T}, itermax=(3 * size(A, 2)))
    x = work.x
    rnorm = work.rnorm
    w = work.w
    zz = work.zz
    idx = work.idx
    
    m, n = size(A)
    @assert size(b) == (m,)
    @assert size(x) == (n,)
    @assert size(rnorm) == (m,)
    @assert size(w) == (n,)
    @assert size(zz) == (m,)
    @assert size(idx) == (n,)
    
    iter = 0
    x .= 0
    idx .= 1:n
    
    izmax = 0
    iz2 = n
    iz1 = 1
    nsetp = 0
    npp1 = 1
    
    @label l30
    
    while true
        # QUIT IF ALL COEFFICIENTS ARE ALREADY IN THE SOLUTION.
        # OR IF M COLS OF A HAVE BEEN TRIANGULARIZED. 
        if (iz1 > iz2 || nsetp >= m)
            break
        end
        
        # COMPUTE COMPONENTS OF THE DUAL (NEGATIVE GRADIENT) VECTOR W().
        for iz in iz1:iz2
            j = idx[iz]
            sm = 0
            for l in npp1:m
                sm += A[l, j] * b[l]
            end
            w[j] = sm
        end
        
        # FIND LARGEST POSITIVE W(J).
        wmax = 0
        for iz in iz1:iz2
            j = idx[iz]
            if w[j] > wmax
                wmax = w[j]
                izmax = iz
            end
        end
        
        # IF WMAX .LE. 0. GO TO TERMINATION.
        # THIS INDICATES SATISFACTION OF THE KUHN-TUCKER CONDITIONS.
        if wmax <= 0
            break
        end
        
        iz = izmax
        j = idx[iz]
        
        # THE SIGN OF W(J) IS OK FOR J TO BE MOVED TO SET P.
        # BEGIN THE TRANSFORMATION AND CHECK NEW DIAGONAL ELEMENT TO AVOID
        # NEAR LINEAR DEPENDENCE.
        Asave = A[npp1, j]
        
    end
        
    
end


nnls! (generic function with 4 methods)

In [18]:
n = 5
m = 6
A = rand(m, n)
b = rand(m)
x = Vector{Float64}(n)
rnorm = Vector{Float64}(m)
w = Vector{Float64}(n)
zz = Vector{Float64}(m)
idx = Vector{Int}(n)
work = NNLSWorkspace(x, rnorm, w, zz, idx)

nnls!(work, A, b)
@assert @wrappedallocs(nnls!(work, A, b)) == 0
nnls!(work, A, b)

LoadError: InterruptException: